## 0. Introduction

Ce projet est réalisé par Zakarya El Mimouni et Yassine Boukhateb étudiants en deuxième année du cycle ingénieur de l'ENSAE Paris dans le cadre du cours de Python pour la DataScience.

**Importation des modules et packages**

In [2]:
import requests
import json
import time
import pandas as pd
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

## I. Récupération des données

On récupère les données de 10 000 films grâce à l'API TMDB

In [6]:
# Votre clé d'API TMDb
api_key = '28c6630049f5d468217e4f34963c03a6'

# URL de base de l'API TMDb pour les films populaires
base_url_popular = 'https://api.themoviedb.org/3/movie/popular'

# Nombre total de films à récupérer
total_movies = 10000

# Nombre de films à récupérer par requête
movies_per_request = 20

# Champs de base pour les films
fields = 'id'  # Ajoutez les champs que vous souhaitez récupérer ici

# Liste pour stocker tous les films avec leurs détails
all_movies_details = []

# Nombre de requêtes nécessaires pour obtenir le nombre total de films
num_requests = total_movies // movies_per_request

# Effectuer les requêtes pour récupérer les films
for page in range(1, num_requests + 1):
    params = {
        'api_key': api_key,
        'page': page,
        'fields': fields
    }
    response = requests.get(base_url_popular, params=params)#, timeout=10)  # Définir un délai d'attente de 10 secondes

    if response.status_code == 200:
        movies_data = response.json()
        movies_reco = movies_data.get('results', [])

            # Pour chaque film, récupérer les détails supplémentaires
        for movie in movies_reco:
            movie_id = movie['id']
            movie_details_url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}&append_to_response=credits,keywords'
            movie_details_response = requests.get(movie_details_url)
            if movie_details_response.status_code == 200:
                movie_details = movie_details_response.json()
                all_movies_details.append(movie_details)
                print(f"Film {movie['title']} récupéré avec détails supplémentaires.")
            else:
                print(f"Erreur lors de la récupération des détails pour {movie['title']}. Statut : {movie_details_response.status_code}")
        print(f"Page {page} récupérée. Total de films récupérés : {len(all_movies_details)}")

    else:
        print(f"Erreur lors de la requête pour la page {page}. Statut : {response.status_code}")


    if len(all_movies_details) >= total_movies:
        break

# Enregistrer les données complètes dans un fichier JSON
with open('movies_data_complete.json', 'w', encoding='utf-8') as f:
    json.dump(all_movies_details, f, ensure_ascii=False, indent=4)

print("Récupération des films avec détails complets terminée. Les données ont été enregistrées dans movies_data_complete.json.")

Film Killers of the Flower Moon récupéré avec détails supplémentaires.
Film Robot Apocalypse récupéré avec détails supplémentaires.
Film Freelance récupéré avec détails supplémentaires.
Film Wonka récupéré avec détails supplémentaires.
Film Trolls Band Together récupéré avec détails supplémentaires.
Film Leave the World Behind récupéré avec détails supplémentaires.
Film Leo récupéré avec détails supplémentaires.
Film Good Boy récupéré avec détails supplémentaires.
Film A Man of Reason récupéré avec détails supplémentaires.
Film Oppenheimer récupéré avec détails supplémentaires.
Film Five Nights at Freddy's récupéré avec détails supplémentaires.
Film The Creator récupéré avec détails supplémentaires.
Film The Advent Calendar récupéré avec détails supplémentaires.
Film Rumble Through the Dark récupéré avec détails supplémentaires.
Film Fast X récupéré avec détails supplémentaires.
Film Fast Charlie récupéré avec détails supplémentaires.
Film Merry Little Batman récupéré avec détails supp

On récupère les données qui nous seront utile pour la création de l'interface de l'application.

In [3]:
movies_data=pd.read_json('movies_data_complete.json')
selected_columns=['id','title','original_title','overview','release_date','adult','popularity','vote_average','vote_count','poster_path','runtime','genres','production_companies','production_countries','spoken_languages','credits','keywords']
movies_data=movies_data[selected_columns]
movies_data= movies_data.drop_duplicates(subset='title', keep='first')

In [5]:
# Convertir la colonne 'release_date' en type datetime si ce n'est pas déjà le cas
movies_data['release_date'] = pd.to_datetime(movies_data['release_date'])

# Filtrer les films dont la date est antérieure à '2023-10-18'
movies_data = movies_data[movies_data['release_date'] <= '2023-12-31']
movies_data.reset_index(drop=True).head()

,id,title,original_title,overview,release_date,adult,popularity,vote_average,vote_count,poster_path,runtime,genres,production_companies,production_countries,spoken_languages,credits,keywords
0,466420,Killers of the Flower Moon,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,2023-10-18,False,1145.813,7.622,1479,/dB6Krk806zeqd0YNp2ngQ9zXteH.jpg,206,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...","[{'id': 194232, 'logo_path': '/oE7H93u8sy5vvW5...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 1157, 'name': 'husband wi..."
1,897087,Freelance,Freelance,An ex-special forces operative takes a job to ...,2023-01-05,False,1243.624,6.539,322,/7Bd4EUOqQDKZXA6Od5gkfzRNb0.jpg,108,"[{'id': 28, 'name': 'Action'}, {'id': 35, 'nam...","[{'id': 89171, 'logo_path': '/c3ttVfx0itQzk2vO...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 6285, 'name': 'coup d'eta..."
2,787699,Wonka,Wonka,Willy Wonka – chock-full of ideas and determin...,2023-12-06,False,1059.479,7.271,316,/qhb1qOilapbapxWQn9jtRCMwXJF.jpg,117,"[{'id': 35, 'name': 'Comedy'}, {'id': 10751, '...","[{'id': 174, 'logo_path': '/IuAlhI9eVC9Z8UQWOI...","[{'iso_3166_1': 'GB', 'name': 'United Kingdom'...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 715, 'name': 'chocolate'}..."
3,901362,Trolls Band Together,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...",2023-10-12,False,920.108,7.172,379,/bkpPTZUdq31UGDovmszsg2CchiI.jpg,92,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...","[{'id': 521, 'logo_path': '/kP7t6RwGz2AvvTkvnI...","[{'iso_3166_1': 'US', 'name': 'United States o...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 1, 'id': ...","{'keywords': [{'id': 3490, 'name': 'pop star'}..."
4,1075794,Leo,Leo,Jaded 74-year-old lizard Leo has been stuck in...,2023-11-17,False,780.064,7.609,597,/pD6sL4vntUOXHmuvJPPZAgvyfd9.jpg,102,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'id': 878, 'logo_path': '/e2AZdsQdkhN0qJhoN4...","[{'iso_3166_1': 'AU', 'name': 'Australia'}, {'...","[{'english_name': 'English', 'iso_639_1': 'en'...","{'cast': [{'adult': False, 'gender': 2, 'id': ...","{'keywords': [{'id': 1517, 'name': 'classroom'..."


On contrôle qu'il n'y pas de variables avec des informations manquantes. On fait le choix d'enlever le film dès lors qu'il manque une info c'est assez restrictif A FAIRE PLUS TARD

In [8]:
movies_data.isnull().sum()


id                      0
title                   0
original_title          0
overview                0
release_date            0
adult                   0
popularity              0
vote_average            0
vote_count              0
poster_path             0
runtime                 0
genres                  0
production_companies    0
production_countries    0
spoken_languages        0
credits                 0
keywords                0
dtype: int64

In [7]:
movies_data.dropna(axis=0,inplace=True)

Pour notre modèle on conserve que les films dont les features ci dessous existent ça sera plus crédible

In [9]:
movies_reco=movies_data[["id","title","overview","genres","keywords","credits"]]

On nettoie les colonnes genres, keywords, credits

In [10]:
movies_reco.loc[0].genres

[{'id': 80, 'name': 'Crime'},
 {'id': 18, 'name': 'Drama'},
 {'id': 36, 'name': 'History'}]

In [11]:
def convert(text):
    L = []
    for i in text:
        L.append(i["name"]) 
    return L 
movies_reco['genres'] = movies_reco['genres'].apply(convert)
movies_reco.head()

/var/folders/8w/8q5hgk2d2wnfsvdq9brjz71h0000gn/T/ipykernel_28423/202457164.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reco['genres'] = movies_reco['genres'].apply(convert)


,id,title,overview,genres,keywords,credits
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,"[Crime, Drama, History]","{'keywords': [{'id': 1157, 'name': 'husband wi...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
1,897087,Freelance,An ex-special forces operative takes a job to ...,"[Action, Comedy]","{'keywords': [{'id': 6285, 'name': 'coup d'eta...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
2,787699,Wonka,Willy Wonka – chock-full of ideas and determin...,"[Comedy, Family, Fantasy]","{'keywords': [{'id': 715, 'name': 'chocolate'}...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
3,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...","[Animation, Family, Music, Fantasy, Comedy]","{'keywords': [{'id': 3490, 'name': 'pop star'}...","{'cast': [{'adult': False, 'gender': 1, 'id': ..."
4,1075794,Leo,Jaded 74-year-old lizard Leo has been stuck in...,"[Animation, Comedy, Family]","{'keywords': [{'id': 1517, 'name': 'classroom'...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."


In [12]:
movies_reco['keywords'] = movies_reco['keywords'].apply(lambda x: x['keywords'] if isinstance(x, dict) and 'keywords' in x else None) # regarder Robot Apocalypse Keywords
movies_reco.head()

/var/folders/8w/8q5hgk2d2wnfsvdq9brjz71h0000gn/T/ipykernel_28423/2707879241.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  movies_reco['keywords'] = movies_reco['keywords'].apply(lambda x: x['keywords'] if isinstance(x, dict) and 'keywords' in x else None) # regarder Robot Apocalypse Keywords


,id,title,overview,genres,keywords,credits
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,"[Crime, Drama, History]","[{'id': 1157, 'name': 'husband wife relationsh...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
1,897087,Freelance,An ex-special forces operative takes a job to ...,"[Action, Comedy]","[{'id': 6285, 'name': 'coup d'etat'}, {'id': 7...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
2,787699,Wonka,Willy Wonka – chock-full of ideas and determin...,"[Comedy, Family, Fantasy]","[{'id': 715, 'name': 'chocolate'}, {'id': 4344...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
3,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...","[Animation, Family, Music, Fantasy, Comedy]","[{'id': 3490, 'name': 'pop star'}, {'id': 5301...","{'cast': [{'adult': False, 'gender': 1, 'id': ..."
4,1075794,Leo,Jaded 74-year-old lizard Leo has been stuck in...,"[Animation, Comedy, Family]","[{'id': 1517, 'name': 'classroom'}, {'id': 434...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."


In [13]:
movies_reco = movies_reco[movies_reco['keywords'].apply(lambda x: isinstance(x, list) and len(x) > 0)]
movies_reco["keywords"]=movies_reco["keywords"].apply(convert)
movies_reco.head()

,id,title,overview,genres,keywords,credits
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,"[Crime, Drama, History]","[husband wife relationship, based on novel or ...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
1,897087,Freelance,An ex-special forces operative takes a job to ...,"[Action, Comedy]","[coup d'etat, dictatorship, political assassin...","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
2,787699,Wonka,Willy Wonka – chock-full of ideas and determin...,"[Comedy, Family, Fantasy]","[chocolate, musical, prequel]","{'cast': [{'adult': False, 'gender': 2, 'id': ..."
3,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...","[Animation, Family, Music, Fantasy, Comedy]","[pop star, brother, musical, sequel, based on ...","{'cast': [{'adult': False, 'gender': 1, 'id': ..."
4,1075794,Leo,Jaded 74-year-old lizard Leo has been stuck in...,"[Animation, Comedy, Family]","[classroom, musical, bucket list]","{'cast': [{'adult': False, 'gender': 2, 'id': ..."


In [14]:
movies_reco['cast'] = movies_reco['credits'].apply(lambda x: x.get('cast') if isinstance(x, dict) else None)
movies_reco['crew'] = movies_reco['credits'].apply(lambda x: x.get('crew') if isinstance(x, dict) else None)
movies_reco.drop("credits", axis=1,inplace=True)
movies_reco.reset_index(drop=True,inplace=True)
movies_reco.head()

,id,title,overview,genres,keywords,cast,crew
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,"[Crime, Drama, History]","[husband wife relationship, based on novel or ...","[{'adult': False, 'gender': 2, 'id': 6193, 'kn...","[{'adult': False, 'gender': 2, 'id': 27, 'know..."
1,897087,Freelance,An ex-special forces operative takes a job to ...,"[Action, Comedy]","[coup d'etat, dictatorship, political assassin...","[{'adult': False, 'gender': 2, 'id': 56446, 'k...","[{'adult': True, 'gender': 2, 'id': 997, 'know..."
2,787699,Wonka,Willy Wonka – chock-full of ideas and determin...,"[Comedy, Family, Fantasy]","[chocolate, musical, prequel]","[{'adult': False, 'gender': 2, 'id': 1190668, ...","[{'adult': False, 'gender': 2, 'id': 1299, 'kn..."
3,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...","[Animation, Family, Music, Fantasy, Comedy]","[pop star, brother, musical, sequel, based on ...","[{'adult': False, 'gender': 1, 'id': 84223, 'k...","[{'adult': False, 'gender': 2, 'id': 10572, 'k..."
4,1075794,Leo,Jaded 74-year-old lizard Leo has been stuck in...,"[Animation, Comedy, Family]","[classroom, musical, bucket list]","[{'adult': False, 'gender': 2, 'id': 19292, 'k...","[{'adult': False, 'gender': 0, 'id': 7963, 'kn..."


In [15]:
def convert3(text):
    L = []
    counter = 0
    for i in text:
        if counter < 3:
            L.append(i['name'])
        counter+=1
    return L 
movies_reco["cast"]=movies_reco["cast"].apply(convert3)
movies_reco.head()

,id,title,overview,genres,keywords,cast,crew
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,"[Crime, Drama, History]","[husband wife relationship, based on novel or ...","[Leonardo DiCaprio, Lily Gladstone, Robert De ...","[{'adult': False, 'gender': 2, 'id': 27, 'know..."
1,897087,Freelance,An ex-special forces operative takes a job to ...,"[Action, Comedy]","[coup d'etat, dictatorship, political assassin...","[John Cena, Alison Brie, Juan Pablo Raba]","[{'adult': True, 'gender': 2, 'id': 997, 'know..."
2,787699,Wonka,Willy Wonka – chock-full of ideas and determin...,"[Comedy, Family, Fantasy]","[chocolate, musical, prequel]","[Timothée Chalamet, Calah Lane, Keegan-Michael...","[{'adult': False, 'gender': 2, 'id': 1299, 'kn..."
3,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...","[Animation, Family, Music, Fantasy, Comedy]","[pop star, brother, musical, sequel, based on ...","[Anna Kendrick, Justin Timberlake, Camila Cabe...","[{'adult': False, 'gender': 2, 'id': 10572, 'k..."
4,1075794,Leo,Jaded 74-year-old lizard Leo has been stuck in...,"[Animation, Comedy, Family]","[classroom, musical, bucket list]","[Adam Sandler, Bill Burr, Cecily Strong]","[{'adult': False, 'gender': 0, 'id': 7963, 'kn..."


In [16]:
def fetch_director(text):
    L = []
    for i in text:
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L 
movies_reco["crew"]=movies_reco["crew"].apply(fetch_director)
movies_reco.head()

,id,title,overview,genres,keywords,cast,crew
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...,"[Crime, Drama, History]","[husband wife relationship, based on novel or ...","[Leonardo DiCaprio, Lily Gladstone, Robert De ...",[Martin Scorsese]
1,897087,Freelance,An ex-special forces operative takes a job to ...,"[Action, Comedy]","[coup d'etat, dictatorship, political assassin...","[John Cena, Alison Brie, Juan Pablo Raba]",[Pierre Morel]
2,787699,Wonka,Willy Wonka – chock-full of ideas and determin...,"[Comedy, Family, Fantasy]","[chocolate, musical, prequel]","[Timothée Chalamet, Calah Lane, Keegan-Michael...",[Paul King]
3,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for...","[Animation, Family, Music, Fantasy, Comedy]","[pop star, brother, musical, sequel, based on ...","[Anna Kendrick, Justin Timberlake, Camila Cabe...",[Walt Dohrn]
4,1075794,Leo,Jaded 74-year-old lizard Leo has been stuck in...,"[Animation, Comedy, Family]","[classroom, musical, bucket list]","[Adam Sandler, Bill Burr, Cecily Strong]",[Robert Smigel]


Ainsi, les colonnes genres, keywords, cast et crew sont exploitables.

In [17]:
movies_reco['overview'] = movies_reco['overview'].apply(lambda x:x.split())
movies_reco["title_bis"]=movies_reco["title"].apply(lambda x:x.split())
movies_reco.head()

,id,title,overview,genres,keywords,cast,crew,title_bis
0,466420,Killers of the Flower Moon,"[When, oil, is, discovered, in, 1920s, Oklahom...","[Crime, Drama, History]","[husband wife relationship, based on novel or ...","[Leonardo DiCaprio, Lily Gladstone, Robert De ...",[Martin Scorsese],"[Killers, of, the, Flower, Moon]"
1,897087,Freelance,"[An, ex-special, forces, operative, takes, a, ...","[Action, Comedy]","[coup d'etat, dictatorship, political assassin...","[John Cena, Alison Brie, Juan Pablo Raba]",[Pierre Morel],[Freelance]
2,787699,Wonka,"[Willy, Wonka, –, chock-full, of, ideas, and, ...","[Comedy, Family, Fantasy]","[chocolate, musical, prequel]","[Timothée Chalamet, Calah Lane, Keegan-Michael...",[Paul King],[Wonka]
3,901362,Trolls Band Together,"[When, Branch's, brother,, Floyd,, is, kidnapp...","[Animation, Family, Music, Fantasy, Comedy]","[pop star, brother, musical, sequel, based on ...","[Anna Kendrick, Justin Timberlake, Camila Cabe...",[Walt Dohrn],"[Trolls, Band, Together]"
4,1075794,Leo,"[Jaded, 74-year-old, lizard, Leo, has, been, s...","[Animation, Comedy, Family]","[classroom, musical, bucket list]","[Adam Sandler, Bill Burr, Cecily Strong]",[Robert Smigel],[Leo]


In [18]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1
movies_reco['cast'] = movies_reco['cast'].apply(collapse)
movies_reco['crew'] = movies_reco['crew'].apply(collapse)
movies_reco['genres'] = movies_reco['genres'].apply(collapse)
movies_reco['keywords'] = movies_reco['keywords'].apply(collapse)
movies_reco.head()

,id,title,overview,genres,keywords,cast,crew,title_bis
0,466420,Killers of the Flower Moon,"[When, oil, is, discovered, in, 1920s, Oklahom...","[Crime, Drama, History]","[husbandwiferelationship, basedonnovelorbook, ...","[LeonardoDiCaprio, LilyGladstone, RobertDeNiro]",[MartinScorsese],"[Killers, of, the, Flower, Moon]"
1,897087,Freelance,"[An, ex-special, forces, operative, takes, a, ...","[Action, Comedy]","[coupd'etat, dictatorship, politicalassassinat...","[JohnCena, AlisonBrie, JuanPabloRaba]",[PierreMorel],[Freelance]
2,787699,Wonka,"[Willy, Wonka, –, chock-full, of, ideas, and, ...","[Comedy, Family, Fantasy]","[chocolate, musical, prequel]","[TimothéeChalamet, CalahLane, Keegan-MichaelKey]",[PaulKing],[Wonka]
3,901362,Trolls Band Together,"[When, Branch's, brother,, Floyd,, is, kidnapp...","[Animation, Family, Music, Fantasy, Comedy]","[popstar, brother, musical, sequel, basedontoy...","[AnnaKendrick, JustinTimberlake, CamilaCabello]",[WaltDohrn],"[Trolls, Band, Together]"
4,1075794,Leo,"[Jaded, 74-year-old, lizard, Leo, has, been, s...","[Animation, Comedy, Family]","[classroom, musical, bucketlist]","[AdamSandler, BillBurr, CecilyStrong]",[RobertSmigel],[Leo]


In [19]:
movies_reco['label'] = movies_reco['overview']+movies_reco['title_bis']+movies_reco['genres'] + movies_reco['keywords'] + movies_reco['cast'] + movies_reco['crew']
data_model=movies_reco.drop(columns=['overview','genres','keywords','cast','crew',"title_bis"])
data_model['label'] = data_model['label'].apply(lambda x: " ".join(x))
data_model.head()

,id,title,label
0,466420,Killers of the Flower Moon,When oil is discovered in 1920s Oklahoma under...
1,897087,Freelance,An ex-special forces operative takes a job to ...
2,787699,Wonka,Willy Wonka – chock-full of ideas and determin...
3,901362,Trolls Band Together,"When Branch's brother, Floyd, is kidnapped for..."
4,1075794,Leo,Jaded 74-year-old lizard Leo has been stuck in...


In [20]:
ps =PorterStemmer()
def stem(text): # stem("dancing")=>"danc"
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)
data_model["label"]=data_model["label"].apply(stem)
data_model["label"]=data_model["label"].apply(lambda x : x.lower())
data_model.head()

,id,title,label
0,466420,Killers of the Flower Moon,when oil is discov in 1920 oklahoma under osag...
1,897087,Freelance,an ex-speci forc oper take a job to provid sec...
2,787699,Wonka,willi wonka – chock-ful of idea and determin t...
3,901362,Trolls Band Together,"when branch' brother, floyd, is kidnap for hi ..."
4,1075794,Leo,jade 74-year-old lizard leo ha been stuck in t...


In [21]:
data_model.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8302 entries, 0 to 8301
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      8302 non-null   int64 
 1   title   8302 non-null   object
 2   label   8302 non-null   object
dtypes: int64(1), object(2)
memory usage: 194.7+ KB


In [22]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')
vector = cv.fit_transform(data_model['label']).toarray()

In [23]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity

array([[1.        , 0.        , 0.        , ..., 0.01690309, 0.06172134,
        0.        ],
       [0.        , 1.        , 0.03846154, ..., 0.09376145, 0.        ,
        0.03580574],
       [0.        , 0.03846154, 1.        , ..., 0.02344036, 0.0855921 ,
        0.10741723],
       ...,
       [0.01690309, 0.09376145, 0.02344036, ..., 1.        , 0.02608203,
        0.04364358],
       [0.06172134, 0.        , 0.0855921 , ..., 0.02608203, 1.        ,
        0.        ],
       [0.        , 0.03580574, 0.10741723, ..., 0.04364358, 0.        ,
        1.        ]])

In [24]:
similarity.shape

(8302, 8302)

In [25]:
def recommend(movie,m):
    index = data_model[data_model['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:m+1]:
        print(data_model.iloc[i[0]].title)

In [26]:
recommend("Killers of the Flower Moon",2)

Wind River
Thunderheart


In [27]:
import pickle
pickle.dump(data_model.to_dict(),open("movies_dict.pkl","wb"))
pickle.dump(similarity,open("similarity.pkl","wb"))